In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np

In [2]:
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)

In [3]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [4]:
sentence = " seoul is the capital of south korea"
char_set = list(set(sentence))
char_dic = {c : i for i, c in enumerate(char_set)}

In [ ]:
char_dic

{' ': 0,
 'a': 2,
 'c': 11,
 'e': 5,
 'f': 13,
 'h': 9,
 'i': 7,
 'k': 6,
 'l': 4,
 'o': 3,
 'p': 12,
 'r': 1,
 's': 10,
 't': 8,
 'u': 14}

In [5]:
vocab_sz = len(char_dic)
hidden_sz = len(char_dic)
input_sz = len(char_dic)

In [6]:
sen_idx = [char_dic[c] for c in sentence]
x_idx = sen_idx[:-1] # " seoul is the capital of south kore"
x_one_hot = [np.eye(vocab_sz)[x] for x in x_idx]
y_data = sen_idx[1:] # "seoul is the capital of south korea"

In [ ]:
np.eye(vocab_sz)

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0.

In [ ]:
x_one_hot

[array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]),
 array([0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]),
 array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 

In [ ]:
y_data

[10,
 5,
 3,
 14,
 4,
 0,
 7,
 10,
 0,
 8,
 9,
 5,
 0,
 11,
 2,
 12,
 7,
 8,
 2,
 4,
 0,
 3,
 13,
 0,
 10,
 3,
 14,
 8,
 9,
 0,
 6,
 3,
 1,
 5,
 2]

In [7]:
x_train = torch.FloatTensor(np.array(x_one_hot))
y_train = torch.LongTensor(np.array(y_data))
# input으로는 float tensor로 만들어주어야 하고 (GPU 행렬 연산에 최적화 하기 위함)
# output 비교 시 int는 long tensor로 만들어주어야, 비교 가능

In [ ]:
class Rnn(nn.Module):
  def __init__(self, input_size, hidden_size, vocab_size):
    super(Rnn, self).__init__()
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.vocab_size = vocab_size

    self.rnn = nn.RNN(input_size=self.input_size, hidden_size=self.hidden_size, batch_first=True)
    self.linear = nn.Linear(self.hidden_size, self.vocab_size)

  def forward(self, x):
    outputs, _ = self.rnn(x)
    x = self.linear(outputs)

    return x

In [8]:
class LSTM(nn.Module):
  def __init__(self, input_size, hidden_size, vocab_size):
    super(LSTM, self).__init__()
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.vocab_size = vocab_size

    self.lstm = nn.LSTM(input_size=self.input_size, hidden_size=self.hidden_size, batch_first=True)
    self.linear = nn.Linear(self.hidden_size, self.vocab_size)

  def forward(self, x):
    outputs, _ = self.lstm(x)
    x = self.linear(outputs)

    return x

In [9]:
# model = Rnn(input_size=input_sz, hidden_size=hidden_sz, vocab_size=vocab_sz).to(device)
model = LSTM(input_size=input_sz, hidden_size=hidden_sz, vocab_size=vocab_sz).to(device)

In [10]:
criterion = torch.nn.CrossEntropyLoss()

In [11]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [12]:
epochs = 500

for epoch in range(epochs):
  model.train()

  outputs = model(x_train.to(device))
  loss = criterion(outputs.view(-1, hidden_sz), y_train.view(-1).to(device))

  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

  result = outputs.data.numpy().argmax(axis=1)
  result_str = ''.join([char_set[idx] for idx in result])

  if(epoch % 50) == 0 or epoch == epochs - 1:
    print(f'loss : {loss} prediction : {result_str}')

loss : 2.699007987976074 prediction : prpr p pp rrpp rpprrpp r  r rr prrr
loss : 2.572385311126709 prediction :                                    
loss : 2.4592134952545166 prediction :                                    
loss : 2.2036399841308594 prediction : soot            ot      oott     eo
loss : 1.7941806316375732 prediction : soout    th   aaitt     sout   oeea
loss : 1.4015387296676636 prediction : sooul    th  aaaittl    south  oeea
loss : 1.064862847328186 prediction : sooul os th  aaaittl o  south  oeea
loss : 0.7912054657936096 prediction : sooul is th  capital o  south  orea
loss : 0.5902738571166992 prediction : seoul is the capital of south korea
loss : 0.4485430419445038 prediction : seoul is the capital of south korea
loss : 0.3476141095161438 prediction : seoul is the capital of south korea


In [ ]:
result

array([10,  0,  3, 14,  4,  0,  8, 10,  0,  8,  9,  5,  0, 11,  2, 12,  7,
        8,  2,  4,  0,  3, 13,  0, 10,  3, 14,  8,  9,  0,  6,  3,  1,  5,
        2])

In [ ]:
np.squeeze(result)

array([10,  0,  3, 14,  4,  0,  8, 10,  0,  8,  9,  5,  0, 11,  2, 12,  7,
        8,  2,  4,  0,  3, 13,  0, 10,  3, 14,  8,  9,  0,  6,  3,  1,  5,
        2])